In [32]:
from path_example_vars import GCS_PATH

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

import pandas as pd
import numpy as np
import geopandas as gpd
from siuba import *
import json

import shared_utils
import warnings
from path_example_vars import GCS_PATH

from shapely.ops import split, substring, LineString
from calitp_data_analysis import geography_utils

CONVEYAL_GCS_PATH = 'gs://calitp-analytics-data/data-analyses/conveyal_update/'

In [6]:
import conveyal_path_utils

In [14]:
df = conveyal_path_utils.read_conveyal_path_df(f'{GCS_PATH}la_PATHS.csv')

In [11]:
def read_conveyal_path_df(path):

    array_cols = ['routes', 'boardStops', 'alightStops',
           'rideTimes', 'waitTimes', 'feedIds']

    def unpack_conveyal_path_df(df, array_cols = array_cols):

        for col in array_cols:
            df.loc[:,col] = df[col].map(lambda x: x.split('|'))
        return df

    df = pd.read_csv(path)
    df['total_iterations'] = df.query('origin == 0 & destination == 0').nIterations.iloc[0]
    df = df.query('origin == 0 & destination == 1').drop(columns=['group'])
    df.reset_index(drop=True, inplace=True)
    df.index.rename('trip_group_id', inplace=True)
    #  won't be in Conveyal order but maps will be less confusing
    #  can hash other cols if need to match Conveyal trip-level unique id
    df.reset_index(drop=False, inplace=True)
    df = unpack_conveyal_path_df(df)
    return df

In [13]:
df2 = read_conveyal_path_df(f'{GCS_PATH}la_PATHS.csv')

In [17]:
df.equals(df2)

True

In [19]:
df2.head(3)

,trip_group_id,origin,destination,routes,boardStops,alightStops,feedIds,rideTimes,accessTime,egressTime,transferTime,waitTimes,totalTime,nIterations,total_iterations
0,0,0,1,"[209-13168, 804, 805, 90-13168, 96-13168]","[141078, 80128, 80211, 12102, 3611]","[2489, 80122, 80213, 3611, 1574]","[654404a5dad1a760f30e340d, 654404b9dad1a760f30...","[14.0, 18.0, 2.0, 3.0, 46.0]",8.6,7.2,2.3,"[1.4, 4.3, 1.5, 1.9, 2.0]",112.2,2,240
1,1,0,1,"[108-13168, 910-13168, 802, 94-13168]","[7962, 2322, 80211, 30001]","[1539, 65300042, 80201, 11812]","[654404a5dad1a760f30e340d, 654404a5dad1a760f30...","[15.0, 12.0, 26.0, 20.0]",1.4,3.8,9.8,"[1.6, 5.3, 2.4, 5.4]",102.8,56,240
2,2,0,1,"[108-13168, 910-13168, 802, 3163, 164-13168]","[7963, 2322, 80211, 3068903, 16428]","[1539, 65300042, 80201, 3068909, 11812]","[654404a5dad1a760f30e340d, 654404a5dad1a760f30...","[16.0, 12.0, 26.0, 12.0, 8.0]",1.3,3.8,10.5,"[1.7, 5.3, 4.4, 4.6, 2.1]",107.8,18,240


In [20]:
df2 = conveyal_path_utils.add_warehouse_identifiers(df2)

In [23]:
points = pd.read_csv(f'{GCS_PATH}la_points.csv')
points = gpd.GeoDataFrame(points, geometry=gpd.points_from_xy(points.lon, points.lat)
                          , crs=geography_utils.WGS84).to_crs(geography_utils.CA_NAD83Albers_m)
points.geometry = points.buffer(100)

In [26]:
#  get warehouse data
warehouse_data = conveyal_path_utils.get_warehouse_data(df2)
# spatial_routes = conveyal_path_utils.compile_all_spatial_routes(df2, warehouse_data, verbose=False)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did 

In [49]:
for _ix, row in df2.iterrows():
    test_ssr = conveyal_path_utils.shape_segments_from_row(row, warehouse_data, True)
    break    

In [50]:
test_ssr

,feed_key,trip_id,stop_id,stop_sequence,stop_geom,name,shape_id,route_short_name,route_long_name,route_id,segment_geom,stop_pair,trip_group_id,optimal_pct,total_time,xfer_count,route_name
0,edb0fd72b98a7f739407ceca3fae9034,10209000510529-JUNE23,141078,32,POINT (155462.061 -445965.161),LA Metro Bus Schedule,2090051_JUNE23,209,Metro Local Line,209-13168,"LINESTRING (155451.791 -445961.441, 155451.900...","(141078, 2489)",0,0.008333,112.2,4,209
0,beaaee89e671fea3d48124dc23335bed,59052614,80122,11,POINT (160750.294 -439298.745),LA Metro Rail Schedule,804WB_RC_221121,None,Metro E-Line,804,"LINESTRING (160750.580 -439298.986, 160748.045...","(80128, 80122)",0,0.008333,112.2,4,Metro E-Line
0,beaaee89e671fea3d48124dc23335bed,58926335,80211,11,POINT (160763.169 -439295.846),LA Metro Rail Schedule,802EB_190513,None,Metro B-Line,802,"LINESTRING (160759.382 -439300.931, 160839.511...","(80211, 80213)",0,0.008333,112.2,4,Metro B-Line
0,edb0fd72b98a7f739407ceca3fae9034,10090003590454-JUNE23,12102,3,POINT (161917.438 -438595.678),LA Metro Bus Schedule,900359_JUNE23,90,Metro Local Line,90-13168,"LINESTRING (161907.791 -438588.470, 161976.674...","(12102, 3611)",0,0.008333,112.2,4,90
0,edb0fd72b98a7f739407ceca3fae9034,10094002580436-JUNE23,3611,16,POINT (162503.972 -437721.182),LA Metro Bus Schedule,940258_JUNE23,94,Metro Local Line,94-13168,"LINESTRING (162493.765 -437717.075, 162501.659...","(3611, 1574)",0,0.008333,112.2,4,94


In [55]:
def shape_segments_from_row(row, warehouse_data, verbose):

    stop_pairs = list(zip(row.boardStops, row.alightStops))
    
    row_shape_segments = []
    for stop_pair in stop_pairs:
        # print(stop_pair)
        first_filter = warehouse_data['st'].query('stop_id.isin(@stop_pair)')
        # display(first_filter)
        good_trips = first_filter.groupby('trip_id')[['stop_id']].count().reset_index().rename(columns={'stop_id':'n'}).query('n > 1')
        #  TODO count frequency using departure sec?
        better_trips = good_trips.query('n == @good_trips.n.max()')
        # display(good_trips)
        assert better_trips.shape[0] > 0
        trip_with_pair = first_filter.query('trip_id == @better_trips.trip_id.iloc[0]').sort_values('stop_sequence')
        trip_with_pair = trip_with_pair[['feed_key', 'trip_id', 'stop_id', 'stop_sequence']]
        trip_with_pair = trip_with_pair.merge(warehouse_data['stops'][['feed_key', 'stop_id', 'geometry']],
                                                      on = ['feed_key', 'stop_id'])
        trip_with_pair = trip_with_pair.merge(warehouse_data['trips'][['feed_key', 'name', 'trip_id', 'shape_id',
                                                                            'route_short_name', 'route_long_name', 'route_id']],
                                                      on = ['feed_key', 'trip_id'])
        paired_shape = warehouse_data['shapes'].query('feed_key == @trip_with_pair.feed_key.iloc[0] & shape_id == @trip_with_pair.shape_id.iloc[0]')
            
        if not trip_with_pair.stop_id.is_unique:
            if verbose:
                print('warning, trip has duplicate stops at a single stop')
            trip_with_pair = trip_with_pair.drop_duplicates(subset=['stop_id'])
        stop0 =  trip_with_pair.query('stop_sequence == @trip_with_pair.stop_sequence.min()').geometry.iloc[0]
        stop1 =  trip_with_pair.query('stop_sequence == @trip_with_pair.stop_sequence.max()').geometry.iloc[0]
        # display(trip_with_pair)
        # print(stop0, stop1)
        if paired_shape.empty:
            if verbose:
                print('warning, trip has no shape')
            trip_with_pair = trip_with_pair.drop_duplicates(subset=['stop_id'])
            paired_segment = LineString([stop0, stop1])
        # stop0_proj = shape_geom.project(stop0)
        # stop1_proj = shape_geom.project(stop1)
        else:
            shape_geom = paired_shape.geometry.iloc[0]
            stops_proj = [shape_geom.project(stop0), shape_geom.project(stop1)] #  be resillient to looping
            paired_segment = substring(shape_geom, min(stops_proj), max(stops_proj))
        
        trip_with_pair['segment_geom'] = paired_segment
        trip_with_pair.set_geometry('segment_geom')
        trip_with_pair = trip_with_pair.rename(columns={'geometry':'stop_geom'})
        # display(stop_pair)
        trip_with_pair = trip_with_pair.drop_duplicates(subset=['shape_id'])
        trip_with_pair['stop_pair'] = [stop_pair]
        trip_with_pair['trip_group_id'] = row.trip_group_id
        trip_with_pair['optimal_pct'] = row.nIterations / row.total_iterations #  more human-friendly name
        # percentige of time in analysis window that this route was optimal
        trip_with_pair['total_time'] = row.totalTime
        trip_with_pair['xfer_count'] = len(stop_pairs) - 1
        trip_with_seg = gpd.GeoDataFrame(trip_with_pair, geometry='segment_geom', crs=geography_utils.CA_NAD83Albers_m)
        row_shape_segments += [trip_with_seg]
        
        spatial_routes = pd.concat(row_shape_segments)
        spatial_routes['route_name'] = spatial_routes.route_short_name.fillna(spatial_routes.route_long_name)
        
    return spatial_routes

In [56]:
for _ix, row in df2.iterrows():
    test_ssr2 = shape_segments_from_row(row, warehouse_data, True)
    break    

In [57]:
test_ssr.equals(test_ssr2)

True